In [ ]:
# THIS IS THE CODE FOR A FOLDER OF TIFF FILES

#I want to open a folder of tiff files and read each file
import os 
path = 'Drug1_1000nM/' #this is the path to the folder where the tiff files are
files = os.listdir(path) #this will list all the files in the folder 
for file in files: #this will loop over all the files in the folder
    open_tiff = imread(path + file) #this will open the tiff file


In [ ]:
# Loading libraries for the course
import os
import random
import urllib.request                       # Library to download data
import numpy as np                          # Library for array manipulation
import pandas as pd                         # Library to manipulate data frames
import tifffile                             # Library to store numpy arrays as TIFF
import pathlib                              # Library to work with file paths
from pathlib import Path                    # Library to work with file paths
import matplotlib.pyplot as plt             # Library used for plotting
from matplotlib.patches import Rectangle    # Module to plot a rectangle in the image
from matplotlib import animation            # Module to plot animations
import seaborn as sn                        # Library for advanced plotting
import skimage                              # Library for image manipulation (scikit-image)
from skimage.io import imread, imshow       # Module from skimage
from skimage import io, measure, filters, morphology, segmentation, color, feature
from skimage.filters import gaussian, threshold_otsu, difference_of_gaussians
from skimage.morphology import (binary_dilation, binary_erosion, binary_closing, 
                                binary_opening, disk, remove_small_objects)
from skimage.draw import polygon
from skimage.measure import regionprops
from skimage.color import label2rgb
from skimage.segmentation import watershed
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from PIL import Image
import cellpose

%matplotlib inline

from ipywidgets import interactive, HBox, VBox, Layout
import ipywidgets as widgets




In [ ]:
#Metadate we want at the end of the analysis

metadata = {'frame_number':[], 'cell_number':[], 'number_of_viable_cells':[], 'cell_size':[], 'nucleus_size':[], 'protein_concentration':[], 'number_of_mRNA_in_nucleus':[], 'number_of_mRNA_in_cytoplasm':[], 'number_of_transcription_sites':[], 'intensity_of_transcription_sites':[]}


In [ ]:
# List directories
os.listdir()

# Set image path and read images
image_path = "Drug1_1000nM/rep1_FOV1.tif"
images = imread(image_path)

# Printing the shape of the image
print('Original image shape:', images.shape)  # [T, Y, X, C]

# Selecting a frame and a color channel
img = images[0, :, :, 0]
print('Single image shape:', img.shape)  # [Y, X]

# channel 0 - red (nucleus sizes, number of viable cells)
[T, Y, X, C] = images.shape
test_image = images.copy()
threshold = 130

# Extracting R, G, B channels from the first frame
img_R = test_image[1, :, :, 0]
img_G = test_image[1, :, :, 1]
img_B = test_image[1, :, :, 2]

# Plotting R, G, B channels
fig, ax = plt.subplots(1, 3, figsize=(7, 7))

ax[0].imshow(img_R)
ax[0].set(title='R')

ax[1].imshow(img_G)
ax[1].set(title='G')

ax[2].imshow(img_B)
ax[2].set(title='B')

plt.show()

# Take R channel and binarize it
threshold = 130
img_R_bin = img_R > threshold
img_R_bin = img_R_bin.astype(int)

# Plot binarized image
plt.imshow(img_R_bin)
plt.title('Binarized R channel')
plt.show()

# Using binarized image to get the number of cells and their middle points
# Labeling the cells
img_R_label = measure.label(img_R_bin)

# Counting the number of cells
num_cells = img_R_label.max()
print('Number of cells:', num_cells)

# Plotting the labeled cells
plt.imshow(img_R_label)
plt.title('Labeled cells')
plt.show()

# Finding the center of the cells
props = measure.regionprops(img_R_label)

# Collecting cell areas
cell_areas = [region.area for region in props]
print("Cell areas:", cell_areas)

centers = [prop.centroid for prop in props]
centers = np.array(centers)

# Plotting the centers
plt.imshow(img_R_bin)
plt.scatter(centers[:, 1], centers[:, 0], c='r')
plt.title('Centers of the cells')
plt.show()

# Round centers locations to integers
centers = np.round(centers).astype(int)
print('Centers of the cells:', centers)

# Initialize metadata
metadata = {'time': [], 'cell_number': [], 'x': [], 'y': [], 'Alive': [], 'nuclei_size': [], 'cell_size': []} 

# Append metadata with time, cell number, x, y coordinates
for i in range(num_cells):
    for j in range(T):
        metadata['time'].append(j)
        metadata['cell_number'].append(i + 1)
        metadata['x'].append(centers[i, 1])
        metadata['y'].append(centers[i, 0])

print(len(metadata['time']))
print(len(metadata['cell_number']))
print(len(metadata['x']))
print(len(metadata['y']))

# Function to check if the center points are 0 in the binarized image
def check_centers(image_array, centers):
    result = []
    for c in centers:
        x, y = c
        if image_array[x, y] == 1:
            result.append(1)  # True
        else:
            result.append(0)  # False
    return result

# Testing the check_centers function
img = test_image[20, :, :, 0]
img_bin = img > threshold
img_bin = img_bin.astype(int)
plt.imshow(img_bin)
plt.show()

result = check_centers(img_bin, centers)
print(result)

# Initialize result_all and cell_areas_all lists
result_all = []
cell_areas_all = []

# Process each time frame
for i in range(T):
    img = test_image[i, :, :, 0]
    img_bin = img > threshold
    img_bin = img_bin.astype(int)

    # Plot binarized image
    plt.imshow(img_bin)
    plt.title(f'Binarized R channel at time {i}')
    plt.show()

    # Check if centers are still 0 in the binarized image
    result = check_centers(img_bin, centers)
    result_all.append(result)

    img_R_label = measure.label(img_bin)
    num_cells = img_R_label.max()
    print('Number of cells:', num_cells)

    props = measure.regionprops(img_R_label)
    cell_areas = [region.area for region in props]

    new_vector = np.zeros(len(centers))
    for t in range(sum(np.array(result) == 1)):
        new_vector[np.where(np.array(result) == 1)[0][t]] = cell_areas[t]

    cell_areas_all.append(new_vector)

result_all = np.array(result_all)
cell_areas_all = np.array(cell_areas_all)

# Append results to metadata
for d in range(len(result_all[0])):
    for z in range(T):
        metadata['Alive'].append(result_all[z, d])
        metadata['nuclei_size'].append(cell_areas_all[z, d])
        print(z, d)

# Debug: Print lengths of metadata fields
for key in metadata:
    print(f"Length of {key}: {len(metadata[key])}")


In [ ]:

# Using the green channel to obtain cell size
cell_sizes_all = []
for i in range(T):
    img_G = test_image[i, :, :, 1]
    
    # Apply Gaussian filter
    img_G_blur = gaussian(img_G, sigma=5) #sigma is the standard deviation of the gaussian kernel used for the filtering operation 
    # can i use a different sigma value? what is the best sigma value for this image?
    
    # Threshold the image
    threshold_G = threshold_otsu(img_G_blur) #this threshold is not working well for this image
    threshold_G = 0.2
    img_G_bin = img_G_blur > threshold_G
    

    #i want to plot the image to see the threshold
    plt.imshow(img_G_bin)
    plt.title(f'Binarized G channel at time {i}')
    plt.show()

    # Label the cells
    img_G_label = measure.label(img_G_bin)
    
    # Measure cell sizes
    props_G = measure.regionprops(img_G_label)
    cell_sizes_frame = [prop.area for prop in props_G]
    
    # Create vector for cell sizes aligned with centers
    new_vector = np.zeros(len(centers))
    for t in range(min(len(cell_sizes_frame), len(new_vector))):
        new_vector[t] = cell_sizes_frame[t]
    
    cell_sizes_all.append(new_vector)

cell_sizes_all = np.array(cell_sizes_all)

# Add cell sizes to metadata

for d in range(len(result)):
    for z in range(T):
        metadata['cell_size'].append(cell_sizes_all[z,d])


In [ ]:
metadata_df = pd.DataFrame(metadata)
metadata_df.to_csv('metadata_pilar.csv')

In [ ]:
#NUMBER OF TRANSCRIPTION SITES

#selecting the blue channel
img_B = test_image[1, :, :, 2]

#show the original image with the blue channel showing the transcription sites
plt.imshow(img_B)
plt.title('Original image')
plt.show()

#apply gaussian filter and show the image  
img_B_blur = gaussian(img_B, sigma=5)
plt.imshow(img_B_blur)
plt.title('Blurred image')
plt.show()

#apply difference of gaussians filter and show the image 
img_B_dog = difference_of_gaussians(img_B_blur, low_sigma=5, high_sigma=10)
plt.imshow(img_B_dog)
plt.title('Difference of Gaussians')
plt.show()

#threshold the image to see only the transcription sites
threshold_B = threshold_otsu(img_B_dog)
img_B_bin = img_B_dog > threshold_B
plt.imshow(img_B_bin)
plt.title('Binarized image')
plt.show()

#the transcription sites must be inside the nucleus, so I will use the nucleus mask to remove the transcription sites that are outside the nucleus
#i need to save the nucleus mask as a tiff file in Wictoria's code and then load it here to use it

#EXAMPLE
#load the nucleus mask
nucleus_mask = imread('nucleus_mask.tif')
plt.imshow(nucleus_mask)
plt.title('Nucleus mask')
plt.show()

#apply the nucleus mask to the binarized image
img_B_bin_nucleus = img_B_bin * nucleus_mask
plt.imshow(img_B_bin_nucleus)
plt.title('Binarized image with nucleus mask')
plt.show()

#label the transcription sites
img_B_label = measure.label(img_B_bin_nucleus)
plt.imshow(img_B_label)
plt.title('Labeled transcription sites')
plt.show()

#count the number of transcription sites INSIDE the nucleus
num_transcription_sites = img_B_label.max()
print('Number of transcription sites:', num_transcription_sites)

#measure the intensity of the transcription sites
props_B = measure.regionprops(img_B_label)

#collect the intensity of the transcription sites
transcription_sites_intensity = [region.mean_intensity for region in props_B]

#add the number of transcription sites and the intensity of the transcription sites to the metadata
metadata['number_of_transcription_sites'] = num_transcription_sites



